In [ ]:
# Importação das bibliotecas necessárias para acessar o serviço Groq
# e carregar o modelo de linguagem Llama 3.3 Versatile.
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv()

# Obter a chave da API do Groq a partir das variáveis de ambiente
# Verificar se a chave da API foi definida corretamente
api_key = os.getenv('GROG_API_KEY')
if not api_key:
    raise ValueError("GROG_API_KEY não está definida. Por favor, defina a chave da API no arquivo .env.")

# Inicializar o modelo de linguagem Llama 3.3 Versatile usando a chave da API
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    api_key=api_key
)

In [ ]:
# Exemplo de uso do modelo de linguagem para invocar uma pergunta
prompt = "Qual é a capital da França?"

# Invocar o modelo de linguagem com o prompt
response = llm.invoke(prompt)

# Exibir a resposta do modelo
print(f"Resposta do modelo: {response}")

### Com dados de fontes internas

In [ ]:
# URL do documento PDF que será utilizado como fonte de dados
url = 'https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/m2m_strategy_and_objectives_development.pdf'


In [ ]:
# Importação do carregador de documentos PDF da biblioteca LangChain Community
from langchain_community.document_loaders import PyPDFLoader

# Carregar o documento PDF a partir da URL
# Criar uma variável chamada `loader` que utiliza o PyPDFLoader para carregar o PDF
loader = PyPDFLoader(url)

# Carregar o documento PDF de forma que tenha vários blocos, ou seja, conteúdo por página
pages = []

# Utilizar o método lazy_load do loader para carregar as páginas do PDF
for page in loader.lazy_load():
    pages.append(page)

In [ ]:
# Exibir os metadados
print(f"{pages[0].metadata}\n")

# Explorar o conteúdo do primeiro bloco carregado
print(f"Conteúdo da primeira página: {pages[0].page_content}\n")

In [ ]:
# Criação de uma base de dados vetorial para armazenar as informações do PDF, usando langchain
# Importação do InMemoryVectorStore da biblioteca LangChain Core, que permite armazenar vetores em memória
from langchain_core.vectorstores import InMemoryVectorStore

# Usar o HuggingFaceEmbeddings para gerar embeddings dos textos, ou seja, tranformar o texto em vetores
from langchain_huggingface import HuggingFaceEmbeddings

# Modelo de embeddings do HuggingFace para transformar o texto em vetores, modelo aberto e gratuito
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")
# necessário instalar pip install sentence-transformers para usar HuggingFaceEmbeddings

# Criar uma base de dados vetorial em memória a partir dos documentos carregados
vector_store = InMemoryVectorStore.from_documents(pages, embed_model)

In [ ]:
# Associando a variável ao VectorStore
docs = vector_store

# Realizar uma busca de similaridade no VectorStore para encontrar documentos relevantes
docs = vector_store.similarity_search

# Realizando a busca por similaridade
#docs = vector_store.similarity_search()

# Exemplo de busca por similaridade (PDF em inglês, então a consulta também deve ser em inglês)
docs = vector_store.similarity_search("Objectives Development Process")

# Mostrar os resultados da busca com k=2 para limitar o número de resultados
docs = vector_store.similarity_search("Objectives Development Process", k=2)

# Exibir os resultados da busca
for doc in docs:
    print(f"Conteúdo: {doc.page_content}\n")
    print(f"Metadados: {doc.metadata}\n")

In [ ]:
retriever = vector_store.as_retriever()

from langchain_core.prompts import ChatPromptTemplate

template = """You're a helpful assistant that only gives answer bases on the given context. If the answer is not in the context, say "I don't know".

Context: {context}

Question: {question}
Answer: """

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
# Prompt
prompt = ChatPromptTemplate.from_template(
    "Use the following context to answer the question:\n\n{context}\n\nQuestion: {question}"
)

# Chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# Importação do módulo display e Markdown da biblioteca IPython para exibir resultados formatados
from IPython.display import display, Markdown

In [ ]:
response = chain.invoke("What is the Objectives Development Process?")
display(Markdown(f"**Resposta:** {response}"))

In [ ]:
from langchain_core.tools import tool

def pega_contexto(query: str) -> str:
    """ Pega o contexto baseado em uma pesquisa."""
    retriever = vector_store.as_retriever()
    resultado = retriever.invoke(query)
    return resultado

In [ ]:
def carrega_pdf(url: str):
    loader = PyPDFLoader(url)
    pages = []
    for page in loader.lazy_load():
        pages.append(page)

    vectorstore = InMemoryVectorStore.from_documents(pages, embed_model)
    return vectorstore


In [ ]:
vector_store_agriculture = carrega_pdf('https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/agriculture.pdf')

In [ ]:
vector_store_dengue = carrega_pdf('https://raw.githubusercontent.com/allanspadini/curso-flash-rag/main/dengue.pdf')


In [ ]:
def pega_contexto_agriculture(query: str) -> str:
    """Pega o contexto sobre agricultura baseado em uma pesquisa."""
    retriever = vector_store_agriculture.as_retriever()
    resultado = retriever.invoke(query)
    return resultado

In [ ]:
def pega_contexto_dengue(query: str) -> str:
    """Pega o contexto sobre dengue baseado em uma pesquisa."""
    retriever = vector_store_dengue.as_retriever()
    resultado = retriever.invoke(query)
    return resultado

In [ ]:
tools = [pega_contexto, pega_contexto_agriculture, pega_contexto_dengue]


In [ ]:
pega_contexto_dengue("Cases of dengue we had since the beginning of 2025?")


In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
# Cria o agente sem system_prompt
agente_pdf = create_react_agent(
    model=llm,  # seu LLM
    tools=tools  # suas ferramentas
)

# Se quiser definir o system prompt, use invoke com o texto de instrução no primeiro "message"
resultado = agente_pdf.invoke({
    "messages": [
        ["system", """You're a helpful assistant that only gives answers based on the given context. 
        If the answer is not in the context, say "I don't know".
        - pega_contexto: Tool that returns the context based on the users query if the query is about NASA and space travels.
        - pega_contexto_agriculture: Tool that returns the context based on the users query if the query is about agriculture.
        - pega_contexto_dengue: Tool that returns the context based on the users query if the query is about dengue."""],
        ["user", "What causes dengue?"]
    ]
})

print(resultado)

### Adicionando memóeria ao Chatbot

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, StateGraph, END
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
# Grafo inicial
grafo = StateGraph(MessagesState)
grafo.add_node("assistente", agente_pdf)

In [ ]:
# Adicionando ferramentas ao grafo
grafo.add_node("tools", ToolNode(tools))
grafo.add_edge(START, "assistente")

# Definir conexões e condições
grafo.add_conditional_edges("assistente", tools_condition)
grafo.add_edge("tools", "assistente")
grafo.add_edge("assistente", END)

In [ ]:
memoria = MemorySaver()

In [ ]:
# Compilar o grafo com o checkpointer de memória
app = grafo.compile(checkpointer=memoria)

Image(app.get_graph().draw_mermaid_png())

In [ ]:
# Criando chatbot com memória
def chat_com_memoria(mensagem_usuario: str, thread_id="1", verbose=False):
    config = {"configurable": {"thread_id": thread_id}}
    messages = app.invoke({"messages": [HumanMessage(content=mensagem_usuario)]}, config)
    if verbose:
        for message in messages['messages']:
            message.pretty_print()
    else:
        messages['messages'][-1].pretty_print()

In [ ]:
# Teste
chat_com_memoria(mensagem_usuario="What is the planet NASA is going?", thread_id="2", verbose=False)